## Deliverable 2. Create a Customer Travel Destinations Map.

In [68]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
import sys
sys.path.append("..")

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [46]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bubaque,GW,11.2833,-15.8333,74.26,61,72,19.46,broken clouds
1,1,Punta Arenas,CL,-53.1500,-70.9167,48.31,71,100,14.97,overcast clouds
2,2,Cape Town,ZA,-33.9258,18.4232,78.12,71,0,1.14,clear sky
3,3,Hithadhoo,MV,-0.6000,73.0833,83.03,74,15,7.49,few clouds
4,4,Tatahuicapan,MX,18.2333,-94.7500,67.08,92,5,3.83,clear sky


In [47]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your desired minimum temperature for your trip? "))
max_temp = float(input("What is your desired maximum temperature for your trip? "))

What is your desired minimum temperature for your trip? 65
What is your desired maximum temperature for your trip? 85


In [49]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) &
                                       (city_data_df["Max Temp"] <= max_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bubaque,GW,11.2833,-15.8333,74.26,61,72,19.46,broken clouds
2,2,Cape Town,ZA,-33.9258,18.4232,78.12,71,0,1.14,clear sky
3,3,Hithadhoo,MV,-0.6000,73.0833,83.03,74,15,7.49,few clouds
4,4,Tatahuicapan,MX,18.2333,-94.7500,67.08,92,5,3.83,clear sky
9,9,East London,ZA,-33.0153,27.9116,70.68,89,100,15.84,light rain
...,...,...,...,...,...,...,...,...,...,...
680,680,Ballina,AU,-28.8667,153.5667,73.74,90,100,17.63,light rain
681,681,Saint-Pierre,RE,-21.3393,55.4781,80.89,76,96,6.13,overcast clouds
682,682,Mabaruma,GY,8.2000,-59.7833,71.47,98,96,3.20,overcast clouds
684,684,Madang,PG,-5.2246,145.7966,79.93,88,100,4.94,light rain


In [50]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [51]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bubaque,GW,11.2833,-15.8333,74.26,61,72,19.46,broken clouds
2,2,Cape Town,ZA,-33.9258,18.4232,78.12,71,0,1.14,clear sky
3,3,Hithadhoo,MV,-0.6000,73.0833,83.03,74,15,7.49,few clouds
4,4,Tatahuicapan,MX,18.2333,-94.7500,67.08,92,5,3.83,clear sky
9,9,East London,ZA,-33.0153,27.9116,70.68,89,100,15.84,light rain
...,...,...,...,...,...,...,...,...,...,...
680,680,Ballina,AU,-28.8667,153.5667,73.74,90,100,17.63,light rain
681,681,Saint-Pierre,RE,-21.3393,55.4781,80.89,76,96,6.13,overcast clouds
682,682,Mabaruma,GY,8.2000,-59.7833,71.47,98,96,3.20,overcast clouds
684,684,Madang,PG,-5.2246,145.7966,79.93,88,100,4.94,light rain


In [52]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bubaque,GW,74.26,broken clouds,11.2833,-15.8333,
2,Cape Town,ZA,78.12,clear sky,-33.9258,18.4232,
3,Hithadhoo,MV,83.03,few clouds,-0.6000,73.0833,
4,Tatahuicapan,MX,67.08,clear sky,18.2333,-94.7500,
9,East London,ZA,70.68,light rain,-33.0153,27.9116,
12,Alofi,NU,81.10,overcast clouds,-19.0595,-169.9187,
13,Hermanus,ZA,70.95,clear sky,-34.4187,19.2345,
14,Touros,BR,79.45,broken clouds,-5.1989,-35.4608,
15,Puerto Narino,CO,71.47,overcast clouds,-3.7703,-70.3831,
17,Saint-Philippe,RE,80.06,overcast clouds,-21.3585,55.7679,


In [53]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Cannot find hotel...skipping.")  

Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot fin

In [54]:
# 7. Drop the rows where there is no Hotel Name.
c = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = c.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name


In [55]:
# 8a. Create the output File (CSV)
output_data_file = "weatherpy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [56]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [69]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 30.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))